In [1]:
import numpy as np
import math
import random
import time
from copy import deepcopy
from sklearn.metrics import f1_score
from load_dataset import *
from set_up_training import *
from prepare_data import *
from utils import *
import sys
from execution import NC_run
# from model_dense import *
from model import *

import torch
from torch import nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tensorboardX import SummaryWriter

import torch_geometric as tg

# SEED = random.randint(1,10001)
SEED = 4329
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

import warnings
warnings.filterwarnings('ignore')
# %matplotlib inline

In [2]:
data_name = 'karate' # emails cora citeseer pubmed cs phisic computers photo karate
adam_lr = 0.01
num_train = 1
local_agg_gnn = 'GCN' # GCN SAGE GIN
fitness_mode = 'both_c' # c: connect s: structure both_j both_c
pooling_mode = 'att' # 'mean' mean max att
num_levels = 2 # 1, 2, 3, 4
encoder_layers = 1
hid_dim = 64
overlap = True
cluster_range = 1 # 1
drop_ratio = 0.5 # 0.9
loss_mode = 'all' # nc recon kl all
output_mode = 'ATT' # ATT GCN GAT MEAN MAX LINEAR LSTM NONE

In [3]:
batch_size = None
early_stop = 100
use_attribute = True
do_view = True

#################Parameters for model#################
loc_time = time.localtime()
model_config={'data_name': data_name,
              'num_train': num_train,
              'num_epoch': 2001, # 2001
              'batch_size': batch_size,
              'lr': adam_lr, # 1e-2, 5e-3
              'l2_regularization': 5e-4, #5e-4, 7e-4
              'drop_ratio': drop_ratio,
              'local_agg_gnn': local_agg_gnn,
              'fitness_mode': fitness_mode,
              'pooling_mode': pooling_mode,
              'output_mode': output_mode,
              'num_levels': num_levels,
              'encoder_layers': encoder_layers,
              'hid_dim': hid_dim,
              'overlap': overlap,
              'cluster_range': cluster_range,
              'loss_mode': loss_mode,
              'verbose': 1, 
              'early_stop': early_stop,
              'use_attribute': use_attribute,
              'do_view': do_view,
              'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model',
              'alias':'test_model_{}_{}_{}_{}_{}'.format(loc_time[0], loc_time[1], loc_time[2], loc_time[3], loc_time[4])}

if torch.cuda.is_available():
    model_config['use_cuda'] = True
else:
    model_config['use_cuda'] = False

print('Parameters of Model are:')
for _ in model_config:
    print(_, model_config[_])
    
# set up device
device = torch.device('cuda' if model_config['use_cuda'] else 'cpu')
model_config['device'] = device

Parameters of Model are:
data_name karate
num_train 1
num_epoch 2001
batch_size None
lr 0.01
l2_regularization 0.0005
drop_ratio 0.5
local_agg_gnn GCN
fitness_mode both_c
pooling_mode att
output_mode ATT
num_levels 2
encoder_layers 1
hid_dim 64
overlap True
cluster_range 1
loss_mode all
verbose 1
early_stop 100
use_attribute True
do_view True
model_dir checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model
alias test_model_2021_11_19_10_16
use_cuda True


In [4]:
# load dataset
ls_data = get_NC_dataset(dataset_name=data_name,
                         use_features=use_attribute,
                         SEED=SEED)
# set up training mode
ls_data = set_up_NC_training(ls_data=ls_data, 
                             num_train=num_train, 
                             SEED=SEED)
# prepare data
ls_data = prepare_NC_data(ls_data=ls_data, 
                          batch_size=batch_size, 
                          dense=False,
                          device=device, 
                          SEED=SEED)

norm = tg.transforms.NormalizeFeatures()
for idx, data in enumerate(ls_data):
    ls_data[idx] = norm(data)

is reading karate dataset...


TypeError: set_up_NC_training() got an unexpected keyword argument 'ls_data'

## experiments

In [ ]:
model = AdamGNN(config=model_config,
              feat_dim=ls_data[0].x.shape[1],
              out_dim=ls_data[0].y.unique().size(0))

# release gpu memory
torch.cuda.empty_cache()

model = model.to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), 
                 lr=model_config['lr'],
                 weight_decay=model_config['l2_regularization'])

In [ ]:
NC_run(model=model, 
       optimizer=optimizer, 
       ls_data=ls_data, 
       model_config=model_config, 
       device=device, 
       SEED=SEED)